# **Nile Basin Flash Flood Hotspots Map: How I did it**

### By Annika McGinnis

My goal in this project was to **scrape and visualize data on flooding in the Nile Basin**, which is mapped on a daily basis on the Nile Basin Flash Flood Early Warning System by the Nile Basin Initiative: https://flashfloodalert.nilebasin.org/workspaces/6c407e1b-5d25-4d83-b782-b6c81f8648ee/applications/9a806474-3cdd-447a-8a01-9898d9974bf8. 

# Project Attempt #1: Many unsuccessful attempts to download administrative regions data

First, I tried scraping the **administrative units layer** of the geodatabase, which shows daily flood risk by sub-national administrative region for 9 countries in the Nile Basin. Inspecting the page code in the Network tab, I opened the API link being loaded on the map for the administrative units data. I realized this data was being shown as WMS images like this one: https://nilebasin-dss-data.azurewebsites.net/geoserver/nile2/wms?TRANSPARENT=true&VERSION=1.3.0&REQUEST=GetMap&FORMAT=image/png&CRS=EPSG:3857&SERVICE=WMS&HEIGHT=865&WIDTH=452&LAYERS=admin_cat,hybas_cat&BBOX=2605661.647053178,-630111.515001775,4852508.670218566,3669717.4120734744&TIME=2024-12-13T00:00:00.000Z 

Using chatGPT with the help of my professor, I was able to devise the **WFS (Web Feature Service) link** for the geodatabase from the WMS (Web Map Service) URL. Through this, I also accessed the *WFS Capabilities URL*, which lists all the feature names and descriptions for the map: https://nilebasin-dss-data.azurewebsites.net/geoserver/nile2/wfs?SERVICE=WFS&REQUEST=GetCapabilities 

I tried downloading data from the WFS URL by modifying the URL with one of the features, specifically by adding the <Name> listing for the nile2:country_categories feature in the < FeatureTypeList >. Through visiting the URL, I was able to download the JSON in my browser and on my machine. I viewed it in mapshaper.org and geojson.io and realized the current bounding box was limiting it to one country. I removed the bounding box and confirmed that the file seemed to include all countries. 

_This is the URL I modified to download this layer: https://nilebasin-dss-data.azurewebsites.net/geoserver/nile2/wfs?SERVICE=WFS&VERSION=1.1.0&REQUEST=GetFeature&TYPENAME=nile2:country_categories&SRSNAME=EPSG:4326&OUTPUTFORMAT=application/json_

I then identified the layer containing flood risk data by administrative regions, which is called **nile2:admin_cat**

_I modified the URL as: https://nilebasin-dss-data.azurewebsites.net/geoserver/nile2/wfs?SERVICE=WFS&VERSION=1.1.0&REQUEST=GetFeature&TYPENAME=nile2:admin_cat&SRSNAME=EPSG:4326&OUTPUTFORMAT=application/json_

For about a week, I made many unsuccessful attempts to download this layer using the requests library, because of its large size. 
- Pagination did not work for this particular URL.
- Adding a bounding box was irrelevant because I needed all the geographic data on the layer.
- Code that attempted to run the API request in a loop until it worked failed repeatedly. 

However, I know the URL was valid, because when I added *MAXFEATURES* to the URL, I was sometimes able to download parts of the data, which I verified by uploading it into mapshaper. I ran the below code many times over the course of two days and was able to download up to 200,000 features, but never the full dataset. The number of features I could download differed at various times of day; sometimes it failed completely and sometimes I could download only up to 10,000 features. It's important to note as well that after downloading the data from the API, this code simplifies geometries to a tolerance of 0.01, which also reduces the file size.

In [ ]:
import requests
import geopandas as gpd
import io

# URL to request WFS data
wfs_url = (
    "https://nilebasin-dss-data.azurewebsites.net/geoserver/nile2/wfs?"
    "SERVICE=WFS&VERSION=1.1.0&REQUEST=GetFeature&TYPENAME=nile2:admin_cat"
    "&SRSNAME=EPSG:4326&OUTPUTFORMAT=application/json"
    "&MAXFEATURES=10000"
)

try:
    # Make a GET request to the WFS URL
    response = requests.get(wfs_url, stream=True)
    response.raise_for_status()  # Raise an error for bad responses

    # Load the data into a GeoDataFrame if the request is successful
    gdf = gpd.read_file(io.BytesIO(response.content))

    # Simplify the geometries (adjust tolerance as needed)
    gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.01, preserve_topology=True)

    # Save the simplified GeoDataFrame to a GeoJSON file
    gdf.to_file("simplified_admin_cat3.geojson", driver="GeoJSON")
    print("Simplified GeoJSON file saved.")
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


## Simplifying and visualizing one large downloaded file of administrative regions data

Although the data couldn't be downloaded via requests or in the browser, I was able to download one of the files by clicking File -> Save Page As while the URL was loading. 

The file size was 8.95 GB. It couldn't be opened in VisualStudioCode and took about 8 hours to open in Sublime Text, but froze so I was unable to scroll through it. I downloaded QGIS and was able to load the main URL and see the options for layers, but it also froze when I try to load any layer. I also tried to view smaller segments of the data using Geopandas in a Python Jupyter Notebook, but this also didn’t work. 

Next, I used Python in a Jupyter Notebook to load this large JSON file from my computer and reduce it to around 400 MB by simplifying geometries. This is the code I used: 

In [ ]:
import geopandas as gpd

input_file = "PATH_TO_MY_FILE"
output_file = "simplified_admin_cat.json"

print("Loading GeoJSON file...")
gdf = gpd.read_file(input_file)

print("Simplifying geometries...")
gdf["geometry"] = gdf["geometry"].simplify(tolerance=0.01, preserve_topology=True)

print(f"Saving simplified GeoJSON to {output_file}...")
gdf.to_file(output_file, driver="GeoJSON")

print("Simplification complete!")

After this, I loaded the file into QGIS, checked to see it captured all regions, simplified geometries a bit more and reduced the decimal points of coordinates (set at 15) to 4. 

Since this file was still huge, it could not be uploaded to Datawrapper, and simplifying geometries more would make regions unrecognizable. I looked up other ways to present the data and came across using **tiles**.

I used tippecanoe to convert the simplified Geojson file (still over 400 MB) to a mbtiles file (18 MB):

In [ ]:
!tippecanoe -o admin_cat.mbtiles PATH_TO_MY_FILE.geojson

map.on('load', function () {
    map.addSource('vector-tiles', {
        type: 'vector',
        url: ''PATH_TO_MY_FILE'' // URL to vector tiles
    });
    map.addLayer({
        'id': 'vector-layer',
        'type': 'fill',
        'source': 'vector-tiles',
        'source-layer': 'layer-name',
        'paint': {
            'fill-color': '#888888',
            'fill-opacity': 0.4
        }
    });
});


I attempted to create a mapbox tile layer using code, all of which failed, so I created a Mapbox Studio account, uploaded the mbtiles files there and then got the tile ID that way.

I created an HTML file linking to the mapbox tile ID, and customized it to create this interactive map (see attached file: **mapbox_nilebasinfloodmap.html**). Although all regions were shown, flood risk values for certain regions could only be seen when zooming in. Also, the map could not be automatically updated. 

**At this point, I decided to switch to downloading the *hotspots layer* instead, which shows similar information on flood risk for certain locations but is a much smaller file mapping points rather than geographic regions.**

I decided to try to use the geographic regions data to come up with a boundary for this map. Back in QGIS with my simplified admin_cat file, I dissolved regions and saved the polygon lines in QGIS. Then I exported it as a Geojson file to get the overall boundary shape. I imported this polygon to Datawrapper, but the map was unable to detect any other locations such as countries or cities like it can for other basemaps. Because it would be hard for viewers to understand where this polygon was located in the world and Datawrapper can only allow for one base map at a time, I decided not to use it, but I uploaded it to my final repo for potential use in layering for more complex maps.

# Project Attempt #2: Downloading and visualizing hotspots layer data

Using this layer was much more straightforward due to its manageable size.

## Step 1: Scraping Flood Risk Geodatabase

In "hotspot_clusters_updater.ipynb," I download the layer **nile2:hybas_hotsport_cluster**, which contains all the information for the hotspots, using requests. This is the code I used: 

In [ ]:
import requests

url = "https://nilebasin-dss-data.azurewebsites.net/geoserver/nile2/wfs?SERVICE=WFS&VERSION=1.1.0&REQUEST=GetFeature&TYPENAME=nile2:hybas_hotsport_cluster&SRSNAME=EPSG:4326&OUTPUTFORMAT=application/json"

response = requests.get(url)
data = response.json()

Although this works, it doesn't work all the time (probably due to the high-volume request to the server). So I revised the code to define a function that would run this code up to 10 times until the response came through. It allows for a two-second time break between tries using time.sleep:

In [ ]:
import time

url = "https://nilebasin-dss-data.azurewebsites.net/geoserver/nile2/wfs?SERVICE=WFS&VERSION=1.1.0&REQUEST=GetFeature&TYPENAME=nile2:hybas_hotsport_cluster&SRSNAME=EPSG:4326&OUTPUTFORMAT=application/json"

def get_api_response(url, max_retries=10):
    attempt = 0  
    while attempt < max_retries:
        attempt += 1  
        response = requests.get(url)
        
        if response.status_code == 200:
            return response  
        else:
            time.sleep(2)  

    return None  

response = get_api_response(url)
response

I checked how many of the same location (info for different days) were in the dataset by using one location. I found out there was data for 533 different days in this file.

In [ ]:
first_location = []

for feature in data['features']:
    if feature['geometry']['coordinates'] == [33.3816, 19.5028]:
        first_location.append(feature['properties']['date_time'])

len(first_location)

I made sure the last day includes today's information by running this code: first_location[-1] 

I tried this out for a couple of other locations to ensure that they were also represented the same number of times in the code.

From the JSON data received as the response to my API call, I extracted IDs, coordinates, flood risk and dates for each hotspot and added them to a list of dictionaries in Python:

In [ ]:
all_hotspots = []

for feature in data['features']:
    hotspot_dict = {}
    hotspot_dict['id'] = feature['id']
    hotspot_dict['coordinates'] = feature['geometry']['coordinates']
    hotspot_dict['flood_risk'] = feature['properties']['max']
    hotspot_dict['date'] = feature['properties']['date_time']
    all_hotspots.append(hotspot_dict)

I imported this list of dictionaries to Pandas as a dataframe:

In [ ]:
import pandas as pd

df = pd.json_normalize(all_hotspots)
df

I saved this data as a CSV: 

In [ ]:
df.to_csv("all_hotspots.csv")

## Step 2: Cleaning and Analyzing Data

I checked the most recent date in the data:

In [ ]:
df.sort_values(by='date', ascending=False)

I determined the total number of hotspots, which is currently 50: 

In [ ]:
df[df['date'] == '2024-12-07T00:00:00Z']['date'].value_counts()

I cleaned the date column using regular expressions to remove the letters and digits after the date:

In [ ]:
df['date'] = df['date'].astype(str)
df['date'] = df['date'].str.extract(r'(\d{4}[-]\d{2}[-]\d{2}).+')

I noticed each ID was different, even for the same locations. I tried to split them with regex so that the IDs can be grouped by location. But in doing so, I realized even the end parts aren't consistent for each location but are just chronological, so there's no way to group locations by IDs. I realized I needed to maintain coordinates for grouping.

I added columns in Pandas for latitude and longitude by converting the coordinates column from an embedded list and then splitting the values in this column.

In [ ]:
last_2_weeks['coordinates'] = last_2_weeks['coordinates'].apply(tuple)

last_2_weeks['latitude'] = last_2_weeks['latitude'].astype(float)
last_2_weeks['longitude'] = last_2_weeks['longitude'].astype(float)

last_2_weeks['coordinates'] = last_2_weeks['coordinates'].astype(str)

last_2_weeks[['longitude', 'latitude']] = last_2_weeks['coordinates'].str.strip('()').str.split(', ', expand=True)
last_2_weeks

I started looking into the number of locations per date, which I had earlier determined to be 50, and realized this differed based on time period. There were 45 locations up to June 29, 2024, which increased to 50 locations on June 30, 2024. This means some hotspot locations are occasionally added or changed. I saw the addition in the results from this code: 

In [ ]:
df_dates = df.groupby('date')['date'].value_counts().reset_index(name="locations_per_date")

df_dates[350:400]

I created a sub-dataset (dataframe) including the data only for the last two weeks. First, I filtered the dataframe by the last 700 rows, since there are 50 rows per day.

In [ ]:
last_2_weeks = df.sort_values(by='date', ascending=False).head(700)

However, I realized that if locations are added, the number of rows per day would also change. I then tried to group by latitude and longitude and take the last 14 values for each group.

In [ ]:
last_2_weeks = df.groupby(['longitude', 'latitude']).head(14)

But this also gave me incorrect data, because it appears some locations have also been removed from the list of hotspots, meaning their most recent values were months ago. I looked up another way to do it and came across using datetime and timedelta to specify the specific days I wanted:

In [ ]:
from datetime import datetime, timedelta

df['date'] = pd.to_datetime(df['date'])

today_date = df['date'].max()  
two_weeks_ago = today_date - timedelta(days=14)

last_2_weeks = df[(df['date'] > two_weeks_ago) & (df['date'] <= today_date)]

I calculated the **maximum flood risk on any day in the last two weeks**, converted this series to a dataframe, and merged this dataframe with my two-weeks dataframe to add maximum flood risk as a new column in the dataframe.

In [ ]:
max_risk_last_2_weeks = last_2_weeks.groupby('coordinates')['flood_risk'].max()
max_risk_last_2_weeks_df = max_risk_last_2_weeks.reset_index(name="max_risk_last_2_weeks")

last_2_weeks = last_2_weeks.merge(max_risk_last_2_weeks_df, how='left', on='coordinates')

I conducted a similar process to determine the **total of the flood risk values in the last two weeks** for each location. 

In [ ]:
sum_risk_last_2_weeks = last_2_weeks.groupby('coordinates')['flood_risk'].sum()
sum_risk_last_2_weeks_df = sum_risk_last_2_weeks.reset_index(name="sum_risk_last_2_weeks")

last_2_weeks = last_2_weeks.merge(sum_risk_last_2_weeks_df, how='left', on='coordinates')

Bringing the dataframe back to a Python list, I added in a column called **radius**, setting hotspots with a two weeks aggregate flood risk of 0 to 1 and hotspots with a two weeks aggregate flood risk of greater than 0 to (sum of risk + 1) * 3. I did this so that Datawrapper can recognize the size of all points (even those with a 0 flood risk value) and size by the total risk in the last two weeks.

In [ ]:
last_2_weeks['radius'] = last_2_weeks['sum_risk_last_2_weeks'] + 1

list_of_dicts = last_2_weeks.to_dict(orient='records')

for hotspot in list_of_dicts:
    if hotspot['radius'] == 1:
        hotspot['radius'] = 2
    elif hotspot['radius'] != 1:
        hotspot['radius'] = hotspot['radius']*3

Using the Google Maps API in Python, I **reverse Geocoded** the coordinates to find the names of each hotspot location. Then I add the locations to the dataframe. I change null values to 'Unknown.'

In [ ]:
def reverse_geocode(lat, lng, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        if result['results']:
            return result['results'][0]['formatted_address']
        else:
            return "No address found"
    else:
        return f"Error: {response.status_code}"

In [ ]:
for location in list_of_dicts:
    lat = location['latitude']
    lng = location['longitude']
    address = reverse_geocode(lat, lng, api_key)
    location['address'] = address

(The below was done later once converting back to a dataframe and filtering for today's values)

In [ ]:
today_df['hotspot_name'].fillna("Unknown", inplace=True)

Using datetime in Python, I change the date format in the date column to [Day of the Week], [Month] Day, Year. This was done during a revision of my Jupyter Notebook after I experienced challenges with automatic date formats in Datawrapper tooltips, which kept adding the time as well.

In [ ]:
for hotspot in list_of_dicts:

    clean_date = hotspot['date'].strftime("%A, %B %d, %Y")
    hotspot['date'] = clean_date

I cleaned the address column to get rid of extraneous text before the city and country name using regular expressions in Pandas:

In [ ]:
last_2_weeks['address'] = last_2_weeks['address'].str.extract(r'.{7}\s(.+)')

Finally, I renamed the address column to **hotspot_name** for easier understanding:

In [ ]:
last_2_weeks = last_2_weeks.rename(columns={'address': 'hotspot_name'})

I then created a dataframe only for today's date, first by extracting the first 50 rows from the revised dataframe, which represents the first instance of all 50 locations in the data.

In [ ]:
today_df = last_2_weeks[:50]

But again, I realized this would differ if locations were added or subtracted. I changed the code to this: 

In [ ]:
today_df = last_2_weeks.groupby(['longitude','latitude'], as_index=False).last()

I did a bit of final cleaning of the today dataframe, removing the ID column and reordering the columns so the location name was first:

In [ ]:
today_df = today_df.drop(columns='id')

In [ ]:
column = today_df.pop('hotspot_name')
today_df.insert(0, 'hotspot_name', column)

I saved both dataframes to CSV (comma separated values) files:

In [ ]:
last_2_weeks.to_csv("flood_risk_last_2_weeks.csv", index=False)

today_df.to_csv("flood_risk_today.csv", index=False)

## Step 3: Auto-Updating the Data

I created a Github repo for this project: https://github.com/annikamcginnis/nile_basin_flood_hotspots_daily/

In **Github Actions - Simple Workflow**, I created a .yml file (update.yml) that imports all the libraries I used in my data download and analysis, requesting the daily dataframe ("flood_risk_today.csv") to update three times every day through the Python Jupyter Notebook file "hotspot_clusters_updater.ipynb." Although I only need updated data once per day, the three times request set at different times of the day allows for potential server-side issues with receiving a response at busy times of the day.

For this, I modified the .yml file from jsoma's bad-air-cities tutorial: https://github.com/jsoma/bad-air-cities/blob/main/.github/workflows/update.yml

## Step 4: Creating Datawrapper Map

I created a **Symbol map** in Datawrapper.de, linking an external dataset to the flood_risk_today.csv file on my repo: https://github.com/annikamcginnis/nile_basin_flood_hotspots_daily/flood_risk_today.csv. 

- I sized points by the radius values.
- I colored points by the daily flash flood risk values, on a sliding scale.
- I added a color and size legend.
- I modified tooltips to include information upon hover for each hotspot's location, flood risk today, and maximum flood risk in the last two weeks.

## Step 5: Creating the Map Website

I created an HTML website file (**index.html**) by modifying the index.html file from jsoma's bad-air-cities tutorial: https://github.com/jsoma/bad-air-cities/blob/main/index.html
I inserted the responsive iFrame embed code from my Datawrapper map.
I modified the CSS to extend the map across the screen and tailor the page design (words, font, background color, etc).
I published the index.html file using Github Pages.

**Final map URL: https://annikamcginnis.github.io/nile_basin_flood_hotspots_daily/**

_I wanted to create a search bar where a user could search for a location and be told how close they are to a flash flood hotspot or a place that has experienced flood risk in the last few weeks, but this was outside the capabilities of Datawrapper. I can possibly explore doing this using other mapping approaches in the future._